In [1]:
#Loading Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")

# load the train data
train_data = pd.read_csv('train.csv')
train_data.head(10)

,cust_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,policy_code,acc_now_delinq,tot_coll_amt,tot_cur_bal,application_type_0,application_type_1,emp_length_0,emp_length_1,emp_length_2,emp_length_3,emp_length_4,emp_length_5,emp_length_6,emp_length_7,...,emp_length_9,emp_length_10,emp_length_11,grade_0,grade_1,grade_2,grade_3,grade_4,grade_5,grade_6,home_ownership_0,home_ownership_1,home_ownership_2,home_ownership_3,home_ownership_4,home_ownership_5,initial_list_status_0,initial_list_status_1,purpose_0,purpose_1,purpose_2,purpose_3,purpose_4,purpose_5,purpose_6,purpose_7,purpose_8,purpose_9,purpose_10,purpose_11,purpose_12,purpose_13,pymnt_plan_0,pymnt_plan_1,term_0,term_1,verification_status_0,verification_status_1,verification_status_2,loan_status
0,A1,20000,20000,20000.0,8.18,407.26,200000.0,8.76,0.0,1.0,14.0,0.0,47274,22.3,29.0,18626.77,18626.77,2027.21,2027.21,1373.23,653.98,0.0,0.00,0.0000,407.26,0.0,1,0.0,0.0,304951.0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1
1,A2,7500,7500,7500.0,11.99,249.08,157000.0,19.04,0.0,1.0,8.0,0.0,20482,98.5,27.0,4897.17,4897.17,3485.51,3485.51,2602.83,882.68,0.0,0.00,0.0000,252.47,0.0,1,0.0,0.0,543005.0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1
2,A3,11200,11200,11100.0,14.65,264.40,80000.0,16.74,1.0,2.0,12.0,0.0,6731,38.2,20.0,10410.24,10317.29,1568.17,1554.17,789.76,778.41,0.0,0.00,0.0000,264.40,0.0,1,0.0,421.0,105430.0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1
3,A4,6000,6000,6000.0,11.99,199.26,55000.0,27.93,1.0,1.0,13.0,0.0,9240,48.6,20.0,2068.11,2068.11,4981.50,4981.50,3931.89,1049.61,0.0,0.00,0.0000,199.26,0.0,1,0.0,406.0,56932.0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,1
4,A5,10000,10000,10000.0,8.67,316.47,40000.0,16.53,0.0,0.0,9.0,0.0,10541,53.0,16.0,0.00,0.00,10545.54,10545.54,10000.00,545.54,0.0,0.00,0.0000,5260.89,0.0,1,0.0,0.0,24462.0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,5
5,A6,16000,16000,16000.0,6.68,491.70,85000.0,29.08,0.0,1.0,12.0,1.0,3843,22.9,25.0,11871.30,11871.30,4911.06,4911.06,4128.70,782.36,0.0,0.00,0.0000,491.70,0.0,1,0.0,0.0,352725.0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1
6,A7,3600,3600,3550.0,16.99,128.34,60000.0,34.60,0.0,0.0,14.0,0.0,11051,68.6,27.0,3119.04,3075.72,763.24,752.64,480.96,282.28,0.0,0.00,0.0000,128.34,0.0,1,0.0,289.0,149822.0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1
7,A8,20000,20000,20000.0,15.80,484.24,50000.0,25.89,0.0,0.0,23.0,0.0,34720,28.7,41.0,0.00,0.00,6880.90,6880.90,2096.15,2262.01,0.0,2522.74,25.2274,484.24,0.0,1,0.0,0.0,62840.0,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
8,A9,35000,35000,35000.0,13.99,814.21,125000.0,26.12,0.0,0.0,13.0,0.0,36624,39.6,36.0,33346.70,33346.70,3389.04,3389.04,1653.30,1735.74,0.0,0.00,0.0000,814.21,0.0,1,0.0,763.0,94908.0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1
9,A10,8875,8875,8875.0,17.57,318.95,50000.0,21.15,0.0,1.0,10.0,0.0,15004,58.2,37.0,3486.56,3486.56,7654.51,7654.51,5388.44,2266.07,0.0,0.00,0.0000,318.95,0.0,1,0.0,0.0,31296.0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1


In [2]:
len(train_data)

621165

In [3]:
#Dropping the nan values
train_data = train_data.dropna()

In [4]:
len(train_data)

621165

In [5]:
# independent variable
X = train_data.drop(['cust_id','loan_status'],1)
y = train_data[["loan_status"]].copy()

In [6]:
#Standardizing the data
scaler = MinMaxScaler()

#Splitting the dataset
X_train1, X_val, y_train1, y_val = train_test_split(X,y, test_size=0.2, random_state=7,stratify=y)
X_train1 = scaler.fit_transform(X_train1)
X_val = scaler.transform(X_val)


In [7]:
# get dummies on the dependent variable
y_train1 = pd.get_dummies(data = y_train1, columns = y_train1.columns)
y_val = pd.get_dummies(data = y_val, columns = y_val.columns)

In [8]:
y_train1.head()

,loan_status_0,loan_status_1,loan_status_2,loan_status_3,loan_status_4,loan_status_5,loan_status_6,loan_status_7,loan_status_8,loan_status_9
15093,0,1,0,0,0,0,0,0,0,0
428363,0,1,0,0,0,0,0,0,0,0
24115,0,1,0,0,0,0,0,0,0,0
523030,0,1,0,0,0,0,0,0,0,0
331248,1,0,0,0,0,0,0,0,0,0


In [9]:
#Model Building
#Import Packages
from keras import models
from keras import layers

In [10]:
# sequential model
model = models.Sequential()

# input layer
model.add(layers.Dense(128,input_dim = X_train1.shape[1], activation='relu'))

# hidden layer
model.add(layers.Dense(128, activation='relu'))

# hidden layer
model.add(layers.Dense(64, activation='relu'))

# hidden layer
model.add(layers.Dense(10, activation='softmax'))

# compile the model
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(X_train1, y_train1, batch_size = 64, epochs = 5)

# calculate the accuracy
accuracy = model.evaluate(X_val,y_val)
print(accuracy[1])

Epoch 1/5
7765/7765 [==============================] - 13s 2ms/step - loss: 0.2798 - accuracy: 0.9321
Epoch 2/5
7765/7765 [==============================] - 13s 2ms/step - loss: 0.1629 - accuracy: 0.9610
Epoch 3/5
7765/7765 [==============================] - 12s 2ms/step - loss: 0.1543 - accuracy: 0.9625
Epoch 4/5
7765/7765 [==============================] - 12s 2ms/step - loss: 0.1535 - accuracy: 0.9622
Epoch 5/5
3883/3883 [==============================] - 3s 836us/step - loss: 0.1523 - accuracy: 0.9622
0.9622000455856323


In [12]:
!pip install optimizers

In [20]:
#Optimizing the model
from keras.optimizers import Adam
from keras.layers import BatchNormalization

opt = Adam(lr=0.0001, decay=1e-6)

# sequential model
model_1 = models.Sequential()

# input layer
model_1.add(layers.Dense(256,input_dim = X_train1.shape[1], activation='relu'))

# hidden layer
model_1.add(layers.Dense(256, activation='relu'))
model_1.add(BatchNormalization())

# hidden layer
model_1.add(layers.Dense(64, activation='relu'))

# hidden layer
model_1.add(layers.Dense(10, activation='softmax'))

# compile the model
model_1.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model_1.fit(X_train1,y_train1,batch_size=128, epochs= 15, validation_data=(X_val,y_val))

accuracy_1 = model_1.evaluate(X_val,y_val)
print(accuracy_1[1])

Epoch 1/15
3883/3883 [==============================] - 19s 5ms/step - loss: 0.5122 - accuracy: 0.8690 - val_loss: 0.2128 - val_accuracy: 0.9548
Epoch 2/15
3883/3883 [==============================] - 18s 5ms/step - loss: 0.2070 - accuracy: 0.9537 - val_loss: 0.1857 - val_accuracy: 0.9577
Epoch 3/15
3883/3883 [==============================] - 18s 5ms/step - loss: 0.1830 - accuracy: 0.9584 - val_loss: 0.1759 - val_accuracy: 0.9590
Epoch 4/15
3883/3883 [==============================] - 17s 4ms/step - loss: 0.1719 - accuracy: 0.9603 - val_loss: 0.1699 - val_accuracy: 0.9611
Epoch 5/15
3883/3883 [==============================] - 18s 5ms/step - loss: 0.1642 - accuracy: 0.9617 - val_loss: 0.1619 - val_accuracy: 0.9622
Epoch 6/15
3883/3883 [==============================] - 18s 5ms/step - loss: 0.1578 - accuracy: 0.9628 - val_loss: 0.1619 - val_accuracy: 0.9618
Epoch 7/15
3883/3883 [==============================] - 17s 5ms/step - loss: 0.1547 - accuracy: 0.9633 - val_loss: 0.1581 - val_ac

In [24]:
#Predictions on test data

# load the test data
test_data = pd.read_csv('test.csv')
cust_id=test_data['cust_id'].values
test_data.drop('cust_id',1,inplace=True)

# transformation using scaler
test_data = scaler.transform(test_data)

# predictions
pred=pd.DataFrame(model.predict(test_data))

# create the submission file
status=pred.idxmax(1)

status.head(10)




0    5
1    1
2    1
3    5
4    1
5    1
6    5
7    1
8    1
9    1
dtype: int64

In [26]:
submission=pd.DataFrame({'cust_id':cust_id, 'loan_status': status})  
submission.to_csv('Submission.csv',index=False,columns=['cust_id','loan_status'])